In [ ]:
from __future__ import print_function

In [ ]:
import os
import nltk
import string
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
with open('alice_in_wonderland.txt', 'r') as content_file: content = content_file.read()

In [ ]:
content2 = " ".join("".join([" " if ch in string.punctuation else ch for ch in content]).split())
tokens = nltk.word_tokenize(content2)
tokens = [word.lower() for word in tokens if len(word)>=2]

In [ ]:
N = 3
quads = list(nltk.ngrams(tokens,N))
newl_app = []
for ln in quads:
  newl = " ".join(ln)
  newl_app.append(newl)

In [ ]:
# Vectorizing the words
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
x_trigm = []
y_trigm = []
for l in newl_app:
  x_str = " ".join(l.split()[0:N-1])
  y_str = l.split()[N-1]
  x_trigm.append(x_str)
  y_trigm.append(y_str)
x_trigm_check = vectorizer.fit_transform(x_trigm).todense()
y_trigm_check = vectorizer.fit_transform(y_trigm).todense()

In [ ]:
# Dictionaries from word to integer and integer to word
dictnry = vectorizer.vocabulary_
rev_dictnry = {v:k for k,v in dictnry.items()}
X = np.array(x_trigm_check)
Y = np.array(y_trigm_check)
Xtrain, Xtest, Ytrain, Ytest,xtrain_tg,xtest_tg = train_test_split(X,Y,x_trigm, test_size=0.3,random_state=42)
print("X Train shape",Xtrain.shape, "Y Train shape" , Ytrain.shape)
print("X Test shape",Xtest.shape, "Y Test shape" , Ytest.shape)

X Train shape (17944, 2562) Y Train shape (17944, 2562)
X Test shape (7691, 2562) Y Test shape (7691, 2562)


In [ ]:
# Model Building
from keras.layers import Input,Dense,Dropout
from keras.models import Model
np.random.seed(42)
BATCH_SIZE = 128
NUM_EPOCHS = 100
input_layer = Input(shape = (Xtrain.shape[1],),name="input")
first_layer = Dense(1000,activation='relu',name = "first")(input_layer)
first_dropout = Dropout(0.5,name="firstdout")(first_layer)
second_layer = Dense(800,activation='relu',name="second")(first_dropout)
third_layer = Dense(1000,activation='relu',name="third") (second_layer)
third_dropout = Dropout(0.5,name="thirdout")(third_layer)
fourth_layer = Dense(Ytrain.shape[1],activation='softmax',name ="fourth")(third_dropout)
history = Model(input_layer,fourth_layer)
history.compile(optimizer = "adam",loss="categorical_crossentropy",metrics=["accuracy"])
print (history.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 2562)]            0         
                                                                 
 first (Dense)               (None, 1000)              2563000   
                                                                 
 firstdout (Dropout)         (None, 1000)              0         
                                                                 
 second (Dense)              (None, 800)               800800    
                                                                 
 third (Dense)               (None, 1000)              801000    
                                                                 
 thirdout (Dropout)          (None, 1000)              0         
                                                                 
 fourth (Dense)              (None, 2562)              256456

In [ ]:
# Model Training
history.fit(Xtrain, Ytrain, batch_size=BATCH_SIZE,epochs=NUM_EPOCHS,verbose=1,validation_split = 0.2)


Epoch 1/100
113/113 [==============================] - 29s 244ms/step - loss: 6.3311 - accuracy: 0.0586 - val_loss: 6.0909 - val_accuracy: 0.0607
Epoch 2/100
113/113 [==============================] - 29s 253ms/step - loss: 5.9139 - accuracy: 0.0687 - val_loss: 6.0233 - val_accuracy: 0.0671
Epoch 3/100
113/113 [==============================] - 19s 169ms/step - loss: 5.6961 - accuracy: 0.0843 - val_loss: 6.0166 - val_accuracy: 0.0738
Epoch 4/100
113/113 [==============================] - 19s 169ms/step - loss: 5.4490 - accuracy: 0.1048 - val_loss: 5.9688 - val_accuracy: 0.0869
Epoch 5/100
113/113 [==============================] - 20s 175ms/step - loss: 5.1947 - accuracy: 0.1316 - val_loss: 5.9860 - val_accuracy: 0.1048
Epoch 6/100
113/113 [==============================] - 21s 185ms/step - loss: 4.9398 - accuracy: 0.1552 - val_loss: 6.0226 - val_accuracy: 0.1076
Epoch 7/100
113/113 [==============================] - 19s 170ms/step - loss: 4.7231 - accuracy: 0.1763 - val_loss: 6.0887 -

In [ ]:
# Model Prediction
Y_pred = history.predict(Xtest)
# Sample check on Test data
print ("Prior bigram words", "|Actual", "|Predicted","\n")
for i in range(10):
  print (i,xtest_tg[i],"|",rev_dictnry[np.argmax(Ytest[i])],"|",rev_dictnry[np.argmax(Y_pred[i])])

241/241 [==============================] - 5s 20ms/step
Prior bigram words |Actual |Predicted 

0 and they | re | walked
1 never learnt | it | for
2 matters it | how | good
3 as she | could | could
4 the duchess | what | and
5 pair of | boots | white
6 cattle in | the | the
7 it was | looking | the
8 what else | have | you
9 voice of | the | the
